## ДЗ 2

### Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

#### Сделано

### Задание 2.
найти фичи с наибольшей значимостью, и вывести их

#### Не успела, доделаю на выходных до 24.04

### Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

#### Не успела, доделаю на выходных до 24.04


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import nltk
import re

In [63]:
with open('hw2_corpus.txt', 'r') as f:
    df = pd.DataFrame({'category': t[0], 'text': t[1]} for t in (l.split(' ', 1) for l in f.readlines()))
df['category'] = df['category'].apply(lambda t: re.search(r'\d+', t).group())
df

,category,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
9995,2,A revelation of life in small town America in ...
9996,2,Great biography of a very interesting journali...
9997,1,Interesting Subject; Poor Presentation: You'd ...
9998,1,Don't buy: The box looked used and it is obvio...


In [64]:
from string import punctuation
stopwords = nltk.corpus.stopwords.words('english')
noise = set(stopwords + list(punctuation))
df['text_tokens'] = df['text'].apply(lambda t: [w for w in nltk.word_tokenize(t) if w not in noise])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [65]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
df['text_tokens'] = df['text_tokens'].apply(lambda l: [ lemmatizer.lemmatize(t) for t in l ])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [85]:
from collections import Counter
corpus = [ t for ts in df.text_tokens for t in ts ]
freq_dicts = list(tp[0] for tp in sorted(Counter(corpus).items(), key=lambda x: -x[1]))

freq_dicts = { 
    '<5%': set(freq_dicts[:len(freq_dicts)//20]), 
    '5-20%': set(freq_dicts[len(freq_dicts)//20 : len(freq_dicts)//5]),  
    '>20%': set(freq_dicts[len(freq_dicts)//5:]),
}


In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

def filtered_text(df_, freq_type):
    return [ ' '.join(t for t in tt if t in freq_dicts[freq_type]) for tt in df_['text_tokens'] ]
    
def fit_and_test(freq_type, vec_cls):
    print('Символы частоты: ' + freq_type)
    x_train = filtered_text(df_train, freq_type)
    y_train = df_train['category']
    x_test = filtered_text(df_test, freq_type)
    y_test = df_test['category']
    
    vec = vec_cls(ngram_range=(1, 1))
    clf = LogisticRegression(random_state=42)
    
    bow = vec.fit_transform(x_train)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))
    print(classification_report(pred, y_test))
    
print('CountVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, CountVectorizer)
    
print('TfidfVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, TfidfVectorizer)

CountVectorizer:
Символы частоты: <5%


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           1       0.83      0.83      0.83      1044
           2       0.81      0.82      0.82       956

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.82      0.82      0.82      2000

Символы частоты: 5-20%
              precision    recall  f1-score   support

           1       0.66      0.70      0.68       978
           2       0.70      0.66      0.68      1022

    accuracy                           0.68      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.68      0.68      0.68      2000

Символы частоты: >20%
              precision    recall  f1-score   support

           1       0.72      0.60      0.65      1240
           2       0.48      0.61      0.54       760

    accuracy                           0.60      2000
   macro avg       0.60      0.61      0.60      2000
weighted avg       0.63      

## Выводы по заданию 1:
Наилучший результат получился с использованием наиболее популярных токенов, как с CountVectorizer, так и с TfidfVectorizer. Вероятно, наименее популярные токены хорошо выделяют отдельные документы, но не категорию. 